-  for each line in smetana outputs, generate a triple in a consumer resource model consisting of donor organism, exchange metabolite, and recipient organism
- convert triple into reactions in antimony as save as SBML
- load SBML into MIRA
- load MIRA into pyCIEMSS (specify priors)
- simulate the model in pyCIEMSS (specify initial conditions)

### for each line in smetana outputs, generate a triple in a consumer resource model consisting of donor organism, exchange metabolite, and recipient organism

In [ ]:
import pandas as pd
import tellurium as te
import antimony as at
import simplesbml 
import mira
import libsbml
import numpy as np


In [ ]:
smetana_file = "../data/smetana_results/smetana_results_gurobi_victor_detailed.tsv"

smetana_df = pd.read_csv(smetana_file, sep='\t')

smetana_df

In [ ]:
np.random.seed(0)

reaction_list = []
parameter_list = []
initial_values_set = set()


for index, row in smetana_df.iterrows():
    donor_reaction = f'\t{row.donor} -> {row.compound}; k_d{index}*{row.donor}'
    receiver_reaction = f'\t{row.compound} -> {row.receiver}; k_r{index}*{row.compound}'
    reaction_list.extend([donor_reaction,receiver_reaction])
    parameter_list.extend([f'\tk_d{index}={row.scs}',f'\tk_r{index}={row.mus}'])
    initial_values_set.add(row.donor)
    initial_values_set.add(row.compound)
    initial_values_set.add(row.receiver)

initial_values_string = "\n".join([f"\t{name}={np.random.uniform(0,1)}" for name in initial_values_set])
species_names_string = "\n".join([f'\t{species} is "{species}"' for species in initial_values_set])

reaction_string = "\n".join(reaction_list)
parameter_string = "\n".join(parameter_list)

antimony_file = f"""
model smetana
{reaction_string}
{parameter_string}
{initial_values_string}
{species_names_string}
end
"""

print(antimony_file)

### convert triple into reactions in antimony as save as SBML

In [ ]:
smbl_file = 'smetana_carbstor_consumer_resource.xml'
rr_model = te.loada(antimony_file)
rr_model.exportToSBML(smbl_file)

# simulate and plot
rr_model.simulate(0, 500, 500)
rr_model.plot()

In [ ]:
import libsbml
corrected_smbl_file = 'smetana_carbstor_consumer_resource_corrected.xml'

d = libsbml.readSBMLFromFile(smbl_file)
m = d.getModel()
# for initial in m.getListOfInitialAssignments():
#     print(initial)

for species in m.getListOfSpecies():
    species.name = species.id
    species.initial_concentration = species.initial_amount
    species.has_only_substance_units = False

libsbml.writeSBMLToFile(d,corrected_smbl_file)

### load SBML into MIRA


In [ ]:
from mira.sources.sbml import template_model_from_sbml_file



In [ ]:
smetana_model = template_model_from_sbml_file(corrected_smbl_file)

smetana_model.initials


In [ ]:
from mira.metamodel import Distribution


dist = Distribution(
                    type= "Uniform1",
                    parameters= {"minimum": 0.0,"maximum": 1.0 }     
                     )

for name,parameter in smetana_model.parameters.items():
    if name != "default_compartment":
        parameter.distribution = dist

parameter

In [ ]:
# initial conditions 


### load MIRA into pyCIEMSS (specify priors)
### simulate the model in pyCIEMSS (specify initial conditions)

see: https://github.com/ciemss/pyciemss/blob/main/docs/source/interfaces.ipynb


In [ ]:
import os
import pyciemss
import torch
import pandas as pd
import numpy as np
from typing import Dict, List, Callable

import pyciemss.visuals.plots as plots
import pyciemss.visuals.vega as vega
import pyciemss.visuals.trajectories as trajectories

from pyciemss.integration_utils.intervention_builder import (
    param_value_objective,
    start_time_objective,
    start_time_param_value_objective,
)

smoke_test = "CI" in os.environ



In [ ]:


# load model
# load data

# 

# set parameters
start_time = 0.0
end_time = 500.0
logging_step_size = 1.0
num_samples = 100

In [ ]:
result1 = pyciemss.sample(smetana_model, end_time, logging_step_size, num_samples, start_time=start_time)
display(result1['data'].head())


In [ ]:
result1.keys()

In [ ]:
# Plot results for all states
schema = plots.trajectories(result1["data"], keep=".*_state")
plots.save_schema(schema, "_schema.json")
plots.ipy_display(schema, dpi=150)